In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install wandb -qU
%pip install --no-deps scikeras
%pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.7/214.7 kB 16.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 7.7 MB/s eta 0:00:00


In [ ]:
#imports
import wandb
from wandb.keras import WandbCallback
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
import sklearn
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K
import tensorflow as tf
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import seaborn as sns
from sklearn.metrics import roc_auc_score
import tensorflow_addons as tfa
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import tensorflow
from keras.optimizers import SGD
from keras.layers import Dropout


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


**Data processing **

In [ ]:
#Upload Real Datasets
df_real1 = pd.read_csv('/content/drive/MyDrive/Zach/RealData/Blue Order 1 (Ready_4ML).csv')
df_real2 = pd.read_csv('/content/drive/MyDrive/Zach/RealData/Red Order 1 (REady_4ML).csv')
df_real3 = pd.read_csv('/content/drive/MyDrive/Zach/RealData/White Order 1 (Ready_4ML).csv')

df_real1.rename(columns={'Run Time ': 'Run Time'}, inplace=True)
df_real2.rename(columns={'Run Time ': 'Run Time'}, inplace=True)
df_real3.rename(columns={'Run Time ': 'Run Time'}, inplace=True)


df2 = df_real1.append(df_real2.append(df_real3))

<ipython-input-4-219118522143>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df_real1.append(df_real2.append(df_real3))


In [ ]:
#Upload Simulated Dataset
df1 = pd.read_csv('/content/drive/MyDrive/Zach/mergedata/RWB_SimDataCombined.csv')

# Rename the 'B' column to 'C' in-place
df1.rename(columns={'Run Time ': 'Run Time'}, inplace=True)

df1['type'] = 0 #SimData
df2['type'] = 1 #RealData
df_combined = df1.append(df2)
df_combined

<ipython-input-5-909f6320fecb>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_combined = df1.append(df2)


,Run Time,msg,node,topics,type
0,0.000000,Published Order,/Order,"/new_order, /rosout",0
1,0.235499,VGR got something,/vgr_node,"/diagnostics, /new_order, /provide_empty_loc, ...",0
2,0.288815,VGR is Moving to HBW,/vgr_node,"/diagnostics, /new_order, /provide_empty_loc, ...",0
3,0.300417,HBW Recieved an Order,/hbw_node,"/diagnostics, /hbw_pub, /new_order, /rosout, /...",0
4,18.445745,HBW Picked Up Material,/hbw_node,"/diagnostics, /hbw_pub, /new_order, /rosout, /...",0
...,...,...,...,...,...
33,18.463000,HBW Picked Up Material,/hbw_node,"/diagnostics, /hbw_pub, /new_order, /rosout, /...",1
34,0.265810,VGR is Moving to HBW,/vgr_node,"/diagnostics, /new_order, /provide_empty_loc, ...",1
35,0.254050,VGR got something,/vgr_node,"/diagnostics, /new_order, /provide_empty_loc, ...",1
36,0.248460,HBW Received an Order,/hbw_node,"/diagnostics, /hbw_pub, /new_order, /rosout, /...",1


In [ ]:
df_combined['topics'] = df_combined['topics'].apply(lambda x: x.replace(" ", ""))
df_combined

,Run Time,msg,node,topics,type
0,0.000000,Published Order,/Order,"/new_order,/rosout",0
1,0.235499,VGR got something,/vgr_node,"/diagnostics,/new_order,/provide_empty_loc,/pr...",0
2,0.288815,VGR is Moving to HBW,/vgr_node,"/diagnostics,/new_order,/provide_empty_loc,/pr...",0
3,0.300417,HBW Recieved an Order,/hbw_node,"/diagnostics,/hbw_pub,/new_order,/rosout,/vgr_...",0
4,18.445745,HBW Picked Up Material,/hbw_node,"/diagnostics,/hbw_pub,/new_order,/rosout,/vgr_...",0
...,...,...,...,...,...
33,18.463000,HBW Picked Up Material,/hbw_node,"/diagnostics,/hbw_pub,/new_order,/rosout,/vgr_...",1
34,0.265810,VGR is Moving to HBW,/vgr_node,"/diagnostics,/new_order,/provide_empty_loc,/pr...",1
35,0.254050,VGR got something,/vgr_node,"/diagnostics,/new_order,/provide_empty_loc,/pr...",1
36,0.248460,HBW Received an Order,/hbw_node,"/diagnostics,/hbw_pub,/new_order,/rosout,/vgr_...",1


In [ ]:
# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'species'.
df_combined['topics']= label_encoder.fit_transform(df_combined['topics'])
df_combined['msg']= label_encoder.fit_transform(df_combined['msg'])
df_combined['node']= label_encoder.fit_transform(df_combined['node'])
df_combined

,Run Time,msg,node,topics,type
0,0.000000,25,0,4,0
1,0.235499,37,7,2,0
2,0.288815,39,7,2,0
3,0.300417,21,1,0,0
4,18.445745,14,1,0,0
...,...,...,...,...,...
33,18.463000,14,1,0,1
34,0.265810,39,7,2,1
35,0.254050,37,7,2,1
36,0.248460,20,1,0,1


In [ ]:
#Setting our prediction target and dropping target columns from the main dataset
predict = 'type'
drops = ['Run Time', 'msg', 'node', 'topics']
X = df_combined.drop(predict, axis=1)
y = df_combined[predict]
print(X.head())
print(y.head())

    Run Time  msg  node  topics
0   0.000000   25     0       4
1   0.235499   37     7       2
2   0.288815   39     7       2
3   0.300417   21     1       0
4  18.445745   14     1       0
0    0
1    0
2    0
3    0
4    0
Name: type, dtype: int64


In [ ]:
X_train, X_test, y_train, y_test =  sklearn.model_selection.train_test_split(X, y, test_size=0.3, shuffle=True)

print(f'Train y: {y_train.shape}')
print(f'Train X: {X_train.shape}')
print(f'Test y: {y_test.shape}')
print(f'Test X: {X_test.shape}')

Train y: (159,)
Train X: (159, 4)
Test y: (69,)
Test X: (69, 4)


In [ ]:
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
Aborted!


In [ ]:
# Example sweep configuration
sweep_configuration = {
    'method': 'random',
    'name': 'sweep',
    'metric': {
        'goal': 'maximize',
        'name': 'f1_score'
		},
    'parameters': {
        'batch_size': {'values': [10,15,25,32,50,64,75,100]},
        'epochs': {'values': [50,75,100,125,150,200,250,300]},
        'fc_layer_size': {'values': [7,9,11,13,15,20,25,30,50]},
        'sc_layer_size': {'values': [7,9,11,13,15,20,25,30,50]},
        'lr': {'max': 0.1, 'min': 0.0001},
        'optimizer':{'values': ['adam','SGD','RMSprop','nadam','Adadelta','Adagrad','Adamax','Ftrl','AdamW']},
        'dropout': {'values': [0, 0.3, 0.5]},
     },

}

sweep_id = wandb.sweep(sweep=sweep_configuration, project="NN_Turing", entity="dt_zach")

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: qvw9nylp
Sweep URL: https://wandb.ai/dt_zach/NN_Turing/sweeps/qvw9nylp


In [ ]:
def get_optimizer(lr=1e-3, optimizer="adam"):
    "Select optmizer between adam and sgd with momentum"
    if optimizer.lower() == "adam":
        return tf.keras.optimizers.Adam(learning_rate=lr)
    if optimizer.lower() == "sgd":
        return tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.1)
    if optimizer.lower() == "rmsprop":
        return tf.keras.optimizers.RMSprop(learning_rate=lr)
    if optimizer.lower() == "nadam":
        return tf.keras.optimizers.Nadam(learning_rate=lr)
    if optimizer.lower() == "adadelta":
        return tf.keras.optimizers.Adadelta(learning_rate=lr)
    if optimizer.lower() == "adagrad":
        return tf.keras.optimizers.Adagrad(learning_rate=lr)
    if optimizer.lower() == "adamax":
        return tf.keras.optimizers.Adamax(learning_rate=lr)
    if optimizer.lower() == "ftrl":
        return tf.keras.optimizers.Ftrl(learning_rate=lr)
    if optimizer.lower() == "adamw":
        return tf.keras.optimizers.AdamW(learning_rate=lr)

In [ ]:
def Nerual_Net(fc_layer_size=11):
  # Set the GPU as the backend for Keras
  config = tf.compat.v1.ConfigProto()
  config.gpu_options.allow_growth = True
  sess = tf.compat.v1.Session(config=config)
  tf.compat.v1.keras.backend.set_session(sess)
  # initiate Network
  network = Sequential()

  # input layer that is fully connected with ReLU activation
  network.add(Dense(4, input_dim=4, activation="relu",use_bias=True,bias_initializer="random_normal", kernel_initializer="he_normal"))

  #hidden layers
  network.add(Dense(fc_layer_size, activation="relu", use_bias=True, bias_initializer="random_normal", kernel_initializer="he_normal"))

  #Output layer that is fully connected with sigmoid activation
  network.add(Dense(1, activation="sigmoid",kernel_initializer="glorot_normal"))

  # Compile network


  #return completed network
  return network

In [ ]:
def train(model, batch_size=64, epochs=10, lr=1e-3, optimizer='adam', log_freq=10):

    # Compile model like you usually do.
    tf.keras.backend.clear_session()
    model.compile(optimizer=get_optimizer(lr, optimizer) ,
               loss="binary_crossentropy",
               metrics=["binary_accuracy",
               tfa.metrics.F1Score(num_classes=2,
                                  average='micro',
                                  threshold=0.5),
               tf.keras.metrics.AUC(from_logits=True)
                        ])

    model.fit(X_train,
              y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              callbacks=[WandbCallback()])

In [ ]:
def sweep_train(config_defaults=None):
    # Initialize wandb with a sample project name
    with wandb.init(config=config_defaults):  # this gets over-written in the Sweep

        # Specify the other hyperparameters to the configuration, if any
        wandb.config.architecture_name = "TestLabfab"
        wandb.config.dataset_name = "Labfab"

        # initialize model
        model = Nerual_Net(wandb.config.fc_layer_size,wandb.config.sc_layer_size,wandb.config.dropout)

        train(model,
              wandb.config.batch_size,
              wandb.config.epochs,
              wandb.config.lr,
              wandb.config.optimizer)

In [ ]:
import pprint
pprint.pprint(sweep_configuration)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'f1_score'},
 'name': 'sweep',
 'parameters': {'batch_size': {'values': [10, 15, 25, 32, 50, 64, 75, 100]},
                'dropout': {'values': [0, 0.3, 0.5]},
                'epochs': {'values': [50, 75, 100, 125, 150, 200, 250, 300]},
                'fc_layer_size': {'values': [7, 9, 11, 13, 15, 20, 25, 30, 50]},
                'lr': {'max': 0.1, 'min': 0.0001},
                'optimizer': {'values': ['adam',
                                         'SGD',
                                         'RMSprop',
                                         'nadam',
                                         'Adadelta',
                                         'Adagrad',
                                         'Adamax',
                                         'Ftrl',
                                         'AdamW']},
                'sc_layer_size': {'values': [7,
                                             9,
               

In [ ]:
wandb.agent(sweep_id, function=sweep_train, count=200)

wandb: Agent Starting Run: 9w4iwjzo with config:
wandb: 	batch_size: 15
wandb: 	dropout: 0
wandb: 	epochs: 300
wandb: 	fc_layer_size: 25
wandb: 	lr: 0.04682751321264652
wandb: 	optimizer: SGD
wandb: 	sc_layer_size: 9
wandb: Currently logged in as: jbinwu39 (dt_zach). Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/300
 1/11 [=>............................] - ETA: 1:10 - loss: 0.7528 - binary_accuracy: 0.7333 - f1_score: 0.8000 - auc: 0.4815

wandb: Adding directory to artifact (/content/wandb/run-20230725_194509-9w4iwjzo/files/model-best)... Done. 0.0s


11/11 [==============================] - 9s 181ms/step - loss: 68.6942 - binary_accuracy: 0.5220 - f1_score: 0.2692 - auc: 0.5071 - val_loss: 0.6961 - val_binary_accuracy: 0.4783 - val_f1_score: 0.0000e+00 - val_auc: 0.5000
Epoch 2/300
 1/11 [=>............................] - ETA: 0s - loss: 0.7092 - binary_accuracy: 0.3333 - f1_score: 0.0000e+00 - auc: 0.5000

...

wandb: Adding directory to artifact (/content/wandb/run-20230725_200629-wgqds6ni/files/model-best)... Done. 0.0s


16/16 [==============================] - 2s 104ms/step - loss: 1.2699 - binary_accuracy: 0.5975 - f1_score: 0.5429 - auc: 0.5832 - val_loss: 2.0599 - val_binary_accuracy: 0.4928 - val_f1_score: 0.4615 - val_auc: 0.5114
Epoch 199/200
13/16 [=======================>......] - ETA: 0s - loss: 1.3588 - binary_accuracy: 0.6000 - f1_score: 0.5593 - auc: 0.5830

wandb: Adding directory to artifact (/content/wandb/run-20230725_200629-wgqds6ni/files/model-best)... Done. 0.0s


16/16 [==============================] - 2s 102ms/step - loss: 1.2627 - binary_accuracy: 0.5975 - f1_score: 0.5429 - auc: 0.5821 - val_loss: 2.0527 - val_binary_accuracy: 0.4928 - val_f1_score: 0.4776 - val_auc: 0.5097
Epoch 200/200
16/16 [==============================] - 0s 9ms/step - loss: 1.2529 - binary_accuracy: 0.5849 - f1_score: 0.5217 - auc: 0.5893 - val_loss: 2.0936 - val_binary_accuracy: 0.5072 - val_f1_score: 0.5143 - val_auc: 0.5000


Exception in thread Thread-2783 (_run_job):
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_run.py", line 2131, in _atexit_cleanup
    self._on_finish()
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_run.py", line 2370, in _on_finish
    _ = exit_handle.wait(timeout=-1, on_progress=self._on_progress_exit)
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/lib/mailbox.py", line 298, in wait
    on_probe(probe_handle)
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_run.py", line 2335, in _on_probe_exit
    result = handle.wait(timeout=0)
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/lib/mailbox.py", line 281, in wait
    raise MailboxError("transport failed")
wandb.sdk.lib.mailbox.MailboxError: transport failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/wandb/agents/pyagent.py",

In [ ]:
run = wandb.init(project="NN_Turing", entity="dt_zach")

Testing the best model from Wandb Sweep

In [ ]:
#Build the model
def Neural_Net():

  # initiate Network
  network = Sequential()

  # input layer that is fully connected with ReLU activation
  network.add(Dense(4, input_dim=4, activation="relu",use_bias=True,bias_initializer="random_normal", kernel_initializer="he_normal"))
  network.add(Dropout(0))
  #hidden layers
  network.add(Dense(25, activation="relu", use_bias=True, bias_initializer="random_normal", kernel_initializer="he_normal"))
  network.add(Dropout(0))
  network.add(Dense(20, activation="relu", use_bias=True, bias_initializer="random_normal", kernel_initializer="he_normal"))

  #Output layer that is fully connected with sigmoid activation
  network.add(Dense(1, activation="sigmoid"))


  # Compile network model.
  network.compile(optimizer=get_optimizer(0.02402, 'RMSprop'),
               loss="binary_crossentropy",
               metrics=["binary_accuracy",
               tfa.metrics.F1Score(num_classes=2,
                                  average='micro',
                                  threshold=0.5),
               tf.keras.metrics.AUC(from_logits=False)
                        ])


  #return completed network
  return network

In [ ]:
# Wrap Keras model so it can be used by scikit-learn
neural_network = KerasClassifier(
                                 model=Neural_Net,
                                 epochs=300,
                                 batch_size=100,
                                 verbose=0,
                                 callbacks=[WandbCallback()]
                                 )

K_fold to ???????

In [ ]:
#Setup stratified Kfold for cross validation
seed = 123457
np.random.seed(seed)
kfold = StratifiedKFold(n_splits=20, shuffle=True, random_state=seed)

In [ ]:
#shows how good the model is
predictions = cross_val_predict(neural_network, X_train, y_train, cv=kfold)

In [ ]:
conf_matrix = confusion_matrix(y_train, predictions)
print(conf_matrix, '\n')

conf_matrix_n = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_matrix_n, annot=True, fmt='.2f', cmap='YlGnBu',
            xticklabels=[0,1], yticklabels=[0,1])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Training Data Confusion Matirix with StratifiedKFold')
plt.xticks(rotation=45)
plt.show(block=False)

print('\nF1 Score: {}'.format(
    f1_score(y_train, predictions, average='macro')))
print('Precision: {}'.format(
    precision_score(y_train, predictions, average='weighted'))) #weighted to show AUC-Precision/Recall curve
print('Recall: {}'.format(
    recall_score(y_train, predictions, average='macro')))
print('Accuracy: {}'.format(
    accuracy_score(y_train, predictions, )))

Test data confrimation

In [ ]:
model = Neural_Net()
#Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=300, batch_size=100, callbacks=[WandbCallback()])
test_pred = model.predict(X_test)


In [ ]:
predictions = (model.predict(X_train) > 0.5).astype("int32")

conf_matrix = confusion_matrix(y_train, predictions)
print(conf_matrix, '\n')

conf_matrix_n = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_matrix_n, annot=True, fmt='.2f', cmap='YlGnBu',
            xticklabels=[0,1], yticklabels=[0,1])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Training Data Confusion Matirix')
plt.xticks(rotation=45)
plt.show(block=False)

print('\nF1 Score: {}'.format(
    f1_score(y_train, predictions, average='macro')))
print('Precision: {}'.format(
    precision_score(y_train, predictions, average='weighted'))) #weighted to show AUC-Precision/Recall curve
print('Recall: {}'.format(
    recall_score(y_train, predictions, average='macro')))
print('Accuracy: {}'.format(
    accuracy_score(y_train, predictions, )))

In [ ]:
predictions_test = (model.predict(X_test) > 0.5).astype("int32")

In [ ]:
conf_matrix = confusion_matrix(y_test, predictions_test)
print(conf_matrix, '\n')

conf_matrix_n = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_matrix_n, annot=True, fmt='.2f', cmap='YlGnBu',
            xticklabels=[0,1], yticklabels=[0,1])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Test Data Confusion Matirix')
plt.xticks(rotation=45)
plt.show(block=False)

print('\nF1 Score: {}'.format(
    f1_score(y_test, predictions_test, average='macro')))
print('Precision: {}'.format(
    precision_score(y_test, predictions_test, average='weighted'))) #weighted to show AUC-Precision/Recall curve
print('Recall: {}'.format(
    recall_score(y_test, predictions_test, average='macro')))
print('Accuracy: {}'.format(
    accuracy_score(y_test, predictions_test, )))

In [ ]:
model.layers[0].get_weights()[1]

In [ ]:
#shows how good the model is
predictions = cross_val_predict(neural_network, X_test, y_test, cv=kfold)

In [ ]:
conf_matrix = confusion_matrix(y_test, predictions)
print(conf_matrix, '\n')

conf_matrix_n = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_matrix_n, annot=True, fmt='.2f', cmap='YlGnBu',
            xticklabels=[0,1], yticklabels=[0,1])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Test data Confusion Matirix with StratifiedKFold')
plt.xticks(rotation=45)
plt.show(block=False)

print('\nF1 Score: {}'.format(
    f1_score(y_test, predictions, average='macro')))
print('Precision: {}'.format(
    precision_score(y_test, predictions, average='weighted'))) #weighted to show AUC-Precision/Recall curve
print('Recall: {}'.format(
    recall_score(y_test, predictions, average='macro')))
print('Accuracy: {}'.format(
    accuracy_score(y_test, predictions, )))

---------------------------------------------------------------------
Further Testing with new datasets of real and sim data

In [ ]:
%cd drive/MyDrive/Zach/RealData
%ls

In [ ]:
df_real1 = pd.read_csv('/content/drive/MyDrive/Zach/RealData/Blue Order 2 (Ready4_ML).csv')
df1 = pd.read_csv('/content/drive/MyDrive/Zach/mergedata/blueSIM_test1.csv')

df_real1.rename(columns={'Run Time ': 'Run Time'}, inplace=True)
# Rename the 'B' column to 'C' in-place
df1.rename(columns={'Run Time ': 'Run Time'}, inplace=True)

df2 = df_real1


df1['type'] = 0 #SimData
df2['type'] = 1 #RealData
df_combined = df1.append(df2)
df_combined

In [ ]:
df_combined['topics'] = df_combined['topics'].apply(lambda x: x.replace(" ", ""))
df_combined

In [ ]:
# Import label encoder
from sklearn import preprocessing

# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'species'.
df_combined['topics']= label_encoder.fit_transform(df_combined['topics'])
df_combined['msg']= label_encoder.fit_transform(df_combined['msg'])
df_combined['node']= label_encoder.fit_transform(df_combined['node'])
df_combined

In [ ]:
#Setting our prediction target and dropping target columns from the main dataset
predict = 'type'
drops = ['Run Time', 'msg', 'node', 'topics']
X = df_combined.drop(predict, axis=1)
y = df_combined[predict]
print(X.head())
print(y.head())

In [ ]:
predictions_combined = (model.predict(X) > 0.5).astype("int32")

In [ ]:
conf_matrix = confusion_matrix(y, predictions_combined)
print(conf_matrix, '\n')

conf_matrix_n = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_matrix_n, annot=True, fmt='.2f', cmap='YlGnBu',
            xticklabels=[0,1], yticklabels=[0,1])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Test data Confusion Matirix for Sim&Real data')
plt.xticks(rotation=45)
plt.show(block=False)

print('\nF1 Score: {}'.format(
    f1_score(y, predictions_combined, average='macro')))
print('Precision: {}'.format(
    precision_score(y, predictions_combined, average='weighted'))) #weighted to show AUC-Precision/Recall curve
print('Recall: {}'.format(
    recall_score(y, predictions_combined, average='macro')))
print('Accuracy: {}'.format(
    accuracy_score(y, predictions_combined, )))

-------------------------------------
Testing with only fake data

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/Zach/mergedata/blueSIM_test1.csv')
df1.rename(columns={'Run Time ': 'Run Time'}, inplace=True)
df1['type'] = 0 #SimData


df1['topics'] = df1['topics'].apply(lambda x: x.replace(" ", ""))
df1


# Encode labels in column 'species'.
df1['topics']= label_encoder.fit_transform(df1['topics'])
df1['msg']= label_encoder.fit_transform(df1['msg'])
df1['node']= label_encoder.fit_transform(df1['node'])
df1

#Setting our prediction target and dropping target columns from the main dataset
predict = 'type'
drops = ['Run Time', 'msg', 'node', 'topics']
X = df1.drop(predict, axis=1)
y = df1[predict]
print(X.head())
print(y.head())

predictions_df1 = (model.predict(X) > 0.5).astype("int32")

conf_matrix = confusion_matrix(y, predictions_df1)
print(conf_matrix, '\n')

conf_matrix_n = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_matrix_n, annot=True, fmt='.2f', cmap='YlGnBu',
            xticklabels=[0,1], yticklabels=[0,1])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Sim data ONLY Confusion Matirix')
plt.xticks(rotation=45)
plt.show(block=False)

print('\nF1 Score: {}'.format(
    f1_score(y, predictions_df1, average='macro')))
print('Precision: {}'.format(
    precision_score(y, predictions_df1, average='weighted'))) #weighted to show AUC-Precision/Recall curve
print('Recall: {}'.format(
    recall_score(y, predictions_df1, average='macro')))
print('Accuracy: {}'.format(
    accuracy_score(y, predictions_df1, )))

In [ ]:
predictions_df1

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/Zach/RealData/Blue Order 2 (Ready4_ML).csv')
df1.rename(columns={'Run Time ': 'Run Time'}, inplace=True)
df1['type'] = 1 #RealData


df1['topics'] = df1['topics'].apply(lambda x: x.replace(" ", ""))
df1


# Encode labels in column 'species'.
df1['topics']= label_encoder.fit_transform(df1['topics'])
df1['msg']= label_encoder.fit_transform(df1['msg'])
df1['node']= label_encoder.fit_transform(df1['node'])
df1

#Setting our prediction target and dropping target columns from the main dataset
predict = 'type'
drops = ['Run Time', 'msg', 'node', 'topics']
X = df1.drop(predict, axis=1)
y = df1[predict]
print(X.head())
print(y.head())

predictions_df1 = (model.predict(X) > 0.5).astype("int32")

conf_matrix = confusion_matrix(y, predictions_df1)
print(conf_matrix, '\n')

conf_matrix_n = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_matrix_n, annot=True, fmt='.2f', cmap='YlGnBu',
            xticklabels=[0,1], yticklabels=[0,1])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Real data ONLY Confusion Matirix')
plt.xticks(rotation=45)
plt.show(block=False)

print('\nF1 Score: {}'.format(
    f1_score(y, predictions_df1, average='macro')))
print('Precision: {}'.format(
    precision_score(y, predictions_df1, average='weighted'))) #weighted to show AUC-Precision/Recall curve
print('Recall: {}'.format(
    recall_score(y, predictions_df1, average='macro')))
print('Accuracy: {}'.format(
    accuracy_score(y, predictions_df1, )))

In [ ]:
predictions_df1